In [4]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

In [3]:
# Specify the URL of the website you want to scrape
url = 'https://finance.yahoo.com/quote/TATAPOWER.NS'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the response using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find and extract the desired data from the HTML
    # You can use various methods provided by BeautifulSoup to navigate and search the HTML structure
    
    # Example: Extract all the text from paragraph elements
    paragraphs = soup.find_all('p')
    for paragraph in paragraphs:
        print(paragraph.text)
else:
    print('Failed to retrieve data from the website.')


Failed to retrieve data from the website.


In [7]:
# Specify the URL of the website you want to scrape
url = 'https://finance.yahoo.com/'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the response using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find and extract the desired data from the HTML
    # You can use various methods provided by BeautifulSoup to navigate and search the HTML structure
    
    # Example: Extract data from a table and save it in a CSV file
    table = soup.find('table')  # Assuming the data is in a table
    rows = table.find_all('tr')  # Find all rows in the table
    
    # Create a CSV file and write the data to it
    with open('data.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        # Write the header row if needed
        # header = [header1, header2, ...]
        # writer.writerow(header)
        
        # Write each row of data to the CSV file
        for row in rows:
            # Extract the data from each cell in the row
            cells = row.find_all('td')
            data = [cell.text.strip() for cell in cells]
            
            # Write the data to the CSV file
            writer.writerow(data)
    
    # Alternatively, you can create a DataFrame from the extracted data
    # and perform further operations on it using pandas
    df = pd.read_csv('data.csv')
    print(df.head())  # Display the first few rows of the DataFrame

else:
    print('Failed to retrieve data from the website.')


Failed to retrieve data from the website.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

# Set up the Selenium webdriver
driver = webdriver.Chrome()
url = 'https://finance.yahoo.com/quote/TATAPOWER.NS/history?period1=820454400&period2=1682899200&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true'
driver.get(url)

# Wait for the page to load
time.sleep(5)

# Scroll to the bottom of the page to load more data
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)

# Keep scrolling until the desired number of rows is reached
while len(driver.find_elements('xpath', '//table/tbody/tr')) < 400:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# Get the page source after scrolling
page_source = driver.page_source

# Close the Selenium webdriver
driver.quit()

# Create a BeautifulSoup object with the page source
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table containing the historical data
table = soup.find('table')

# Create empty lists to store the data
dates = []
open_prices = []
high_prices = []
low_prices = []
close_prices = []
adj_close_prices = []
volumes = []

# Extract data from each row of the table
for row in table.tbody.find_all('tr'):
    cols = row.find_all('td')
    if len(cols) == 7:
        date = cols[0].text
        open_price = float(cols[1].text.replace(',', ''))
        high_price = float(cols[2].text.replace(',', ''))
        low_price = float(cols[3].text.replace(',', ''))
        close_price = float(cols[4].text.replace(',', ''))
        adj_close_price = float(cols[5].text.replace(',', ''))
        volume = int(cols[6].text.replace(',', ''))
        
        dates.append(date)
        open_prices.append(open_price)
        high_prices.append(high_price)
        low_prices.append(low_price)
        close_prices.append(close_price)
        adj_close_prices.append(adj_close_price)
        volumes.append(volume)

# Create a DataFrame from the collected data
data = pd.DataFrame({
    'Date': dates,
    'Open': open_prices,
    'High': high_prices,
    'Low': low_prices,
    'Close': close_prices,
    'Adj Close': adj_close_prices,
    'Volume': volumes
})

# Define the file path for the CSV file
csv_file_path = 'stock_data.csv'

# Check if the CSV file exists and delete it if it does
if os.path.exists(csv_file_path):
    os.remove(csv_file_path)

# Save the data as a CSV file
data.to_csv(csv_file_path, index=False)

print("Data saved as CSV:", csv_file_path)
print(data.head())
